## Tutorials
- https://anaconda.org/fpliger/06-interactions/notebook
- http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/widgets.html#userguide-interaction-widgets
- https://rebeccabilbro.github.io/interactive-viz-bokeh/ GapMinder demo
- http://bokeh.pydata.org/en/latest/docs/user_guide/server.html Bokeh Server

# Callbacks
- how are they set up?
- v1

```python
# define a ColumnDataSource (1)
source=ColumnDataSource(data=df) 
# define a normal plot (2) with ColumnDataSource as input (1->2)
p1 = figure(plot_width=500, plot_height=300, logo=None, toolbar_location='right') 
p1.circle(source=source, x='count', y='y', size=20, alpha=0.5)  # CHANGED 
# define callback JS (3) with ColumnDataSource as input and a trigger source.trigger('change') (1->3)
callback = CustomJS(args=dict(source=source), code=""" 
        var data = source.get('data'); 
        var f = cb_obj.get('value') 
        data['y'] = data[f]             
        source.trigger('change'); 
    """) 
# insert the callback call to the Select widget (4) (3->4) 
y_axis = Select(title="Y:", value='item1', options=['item1', 'item2', 'item3'], callback = callback) 
```

- v2

```python
# callback code to be used by all the filter widgets
# requires (source, original_source, grp_select_obj, target_object)
combined_callback_code = """
var data = source.get('data');
var original_data = original_source.get('data');
var grp = grp_select_obj.get('value');
console.log("grp: " + grp);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['grp'].length; ++i) {
        if (( grp === 'ALL' || original_data['grp'][i] === grp)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.trigger('change');
source.trigger('change');
"""

# define the filter widgets, without callbacks for now
grp_select = Select(title="Grp:", value='ALL', options=['ALL','A','B','C'])

# now define the callback objects now that the filter widgets exist
generic_callback = bokeh.models.CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              grp_select_obj=grp_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
grp_select.callback = generic_callback
```

```python
def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var A = amp.value;
        var k = freq.value;
        var phi = phase.value;
        var B = offset.value;
        x = data['x']
        y = data['y']
        for (i = 0; i < x.length; i++) {
                y[i] = B + A*Math.sin(k*x[i]+phi);
            }
        
        source.change.emit();
    """)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude", callback=callback, callback_policy='mouseup')
    callback.args["amp"] = amp_slider

    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency", callback=callback)
    callback.args["freq"] = freq_slider

    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase", callback=callback)
    callback.args["phase"] = phase_slider

    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset", callback=callback)
    callback.args["offset"] = offset_slider

    widgets = WidgetBox(amp_slider, freq_slider, phase_slider, offset_slider)
```

- are there different ways of defining a callback?
- JS vs python callback function

When used with the Bokeh server, widgets can run arbitrary Python code, enabling complex applications. Widgets can also be used without the Bokeh server in standalone HTML documents through the browser’s Javascript runtime.

- Use the CustomJS callback (see JavaScript Callbacks). This will work in standalone HTML documents.
- Use bokeh serve to start the Bokeh server and set up event handlers with .on_change (or for some widgets, .on_click).

All widgets have an .on_change method that takes an attribute name and one or more event handlers as parameters. These handlers are expected to have the function signature, (attr, old, new), where attr refers to the changed attribute’s name, and old and new refer to the previous and updated values of the attribute. .on_change must be used when you need the previous value of an attribute.

Additionally, some widgets, including the button, dropdown, and checkbox, have an .on_click method that takes an event handler as its only parameter. For the Button, this handler is called without parameters. For the other widgets with .on_click, the handler is passed the new attribute value.